In [ ]:
from IPython.display import HTML, clear_output

!pip install -q mujoco dm_control transforms3d
!pip install -q git+https://github.com/kevinzakka/brax
clear_output()

from typing import List

import numpy as np
import mujoco
from brax.tools import mujoco as braxmjc

!git clone -b js-renderer https://github.com/kevinzakka/mujoco.git
%cd mujoco/js/
from mujoco_html import CartesianPose, render
clear_output()

In [ ]:
# Load the the model.
model = mujoco.MjModel.from_xml_path("ant_scene.xml")
data = mujoco.MjData(model)

with open("ant_scene.xml", "r") as f:
    xml_string = f.read()

m = braxmjc.MujocoConverter(
    xml_string,
    ignore_unsupported_joints=True,
    add_collision_pairs=True,
)
config = m.config
config.dt = model.opt.timestep

In [ ]:
# This will contain Cartesian poses of the bodies in the world frame.
frames: List[CartesianPose] = []

# Apply random controls for a 2 second duration.
while data.time < 2.0:
    data.ctrl = np.random.uniform(*model.actuator_ctrlrange.T)
    mujoco.mj_step(model, data)
    frames.append(CartesianPose.from_mjdata(data))

In [ ]:
HTML(render(config, frames))